# Homework 3

## Inference on Graphical Models

In the previous homework, you computed the posterior probabilities of either the cook (C), the butler (B), or both, being a murdered given the choice of weapons (K = knife, P = poison). In addition there is the possibility that a third party may have:
- participated in the murder with one or both of the cook or butler, 
- acted alone in the murder,
- or not participated. 

In this exercise you will construct a Directed Bayesian Graphical Model or belief network for the available evidence and perform inference on some of the variables. 

Inspector Markov has continued her investigation. Additionally Dr. Turing has had time to perform laboratory analysis. Turing reports to the Inspector that there is a chance that a toxic substance may have been used to incapacitate the victim before the stabbing. So, there are now two possible weapons:
- knife alone, 
- knife with a poison. 

Given this evidence, Inspector Markov must update her beliefs. Further, she needs to perform inference to determine if there are likely combinations of suspects and weapons. 

As a first step in creating the belief network, import the packages you will need for this analysis.

In [1]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

The joint probability distribution is:

$$p(B,C,BW,CW,M)$$   
where the letters indicate the following variables;   
$B = $ butler committed the crime, {not murderer, murderer},   
$C = $ cook committed the crime, {not murderer, murderer},    
$BW = $ choice of weapon, {knife, knife with poison}, conditional on butler,  
$CW = $ choice of weapon, {knife, knife with poison}, conditional on cook,   
$M = $ murderer {butler or cook, third party alone, combination of butler or cook and third party}.    

Keeping in mind it is possible the cook, the butler and the third party could be guilty, and that participation in the crime in any capacity constitutes guilt, the distribution can be factorized in the following manner:

$$p(B,C,BW,CW,M) = p(B)\ p(C)\ p(BW\ |\ B)\ p(CW\ |\ C)\ p(M\ |\ BW,CW, C, B)$$  

Now you will need to define the skeleton of the graph. Given the independency relationships of the factorized probability distribution define the skeleton of the model (`m_model`) using the `BayesianModel` function.

>**Hint:** Using paper and pencil make a sketch of the graph before you commit your skeleton structure to code. This structure will be identical to the previous homework.  

>**WARNING:** The distribution and its factorization used for this problem are quite different from previous assignments. 

In [7]:
m_model = BayesianModel([('B', 'BW'), ('C', 'CW'), 
                        ('BW', 'M'), ('CW', 'M'), ('B','M'),('C','M')])

Your next step to create you model is to define the CDP for each independent variable using the `TabularCDP` function. The prior is that butlers are more likely to be murderers than cooks. The tables for these variables are:    


$p(B)$   

| Case | p |
|---|---|
|B0 | 0.2 |
|B1 | 0.8 |    

$p(C)$   

| Case | p |
|---|---|
|C0 | 0.7 |
|C1 | 0.3 |


Using the above tables define the CDPs. Make sure you use variable names consistent with your model.

In [8]:
CDP_B = TabularCPD(variable='B', variable_card=2, values=[[0.2, 0.8]])
CDP_C = TabularCPD(variable='C', variable_card=2, values=[[0.7, 0.3]])
print(CDP_B)
print(CDP_C)

╒═════╤═════╕
│ B_0 │ 0.2 │
├─────┼─────┤
│ B_1 │ 0.8 │
╘═════╧═════╛
╒═════╤═════╕
│ C_0 │ 0.7 │
├─────┼─────┤
│ C_1 │ 0.3 │
╘═════╧═════╛


Next, define the variables $BW$ and $CW$, the conditional probabilities of choice of weapon given butler or cook. These suspects have difference likely choices of weapons. The conditional probability tables for these variables are:

$$p(BW)$$



| | B0| B1 |
|---|---|---|
|BW0 | 0.5 | 0.9 | 
|BW1 | 0.5 | 0.1 | 


Here;  
BW0 = Butler and knife,   
BW1 = Butler and knife with poison.


$$p(CW)$$

| | C0| C1 |
|---|---|---|
|CW0 | 0.5 | 0.3 | 
|CW1 | 0.5 | 0.7 | 


Here;  
CW0 = Cook and knife,   
CW1 = Cook and knife with poison.

>**Note:** when the butler or cook are not a murderer we have a uniform on uninformative distribution. 

Give the above tables define the CDPs. 

In [9]:
CDP_BW = TabularCPD(variable='BW', variable_card=2, values=[[0.5, 0.9],[0.5,0.1]],
                   evidence=['B'], evidence_card=[2])
print(CDP_BW)
CDP_CW = TabularCPD(variable='CW', variable_card=2, values=[[0.5, 0.3],[0.5,0.7]],
                   evidence=['C'], evidence_card=[2])
print(CDP_CW)

╒══════╤═════╤═════╕
│ B    │ B_0 │ B_1 │
├──────┼─────┼─────┤
│ BW_0 │ 0.5 │ 0.9 │
├──────┼─────┼─────┤
│ BW_1 │ 0.5 │ 0.1 │
╘══════╧═════╧═════╛
╒══════╤═════╤═════╕
│ C    │ C_0 │ C_1 │
├──────┼─────┼─────┤
│ CW_0 │ 0.5 │ 0.3 │
├──────┼─────┼─────┤
│ CW_1 │ 0.5 │ 0.7 │
╘══════╧═════╧═════╛


Finally, you must define the probability of the murder which are coded:

- **M0:** Named party (cook or butler or both) with no third unnamed party, 
- **M1:** Only the third unnamed party alone (not cook and not butler), 
- **M2:** Named party (butler or cook or both) and unnamed party together. 

This CDP is conditional on BK, CK, C and B. Since there are three possible guilty parties here are 48 possible states; $N_{cook_or_butler} *  N_{BK} * N_{CK} * N_{M} = 2 * 2 * 2 * 3 = 48$ as shown here:

| | p | p | p | p | p | p | p | p | p | p | p | p |  p | p | p | p |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|| CW0 | CW0 | CW0 | CW0 | CW0 | CW0 | CW0 | CW0 | CW1 | CW1 | CW1 | CW1 | CW1 | CW1 | CW1 | CW1 |
|| BW0 | BW0 | BW0 | BW0 | BW1 | BW1 | BW1 | BW1 | BW0 | BW0 | BW0 | BW0 | BW1 | BW1 | BW1 | BW1 |
|| C0 | C0 | C1 | C1 | C0 | C0 | C1 | C1| C0 | C0 | C1 | C1| C0 | C0 | C1 | C1 |
|| B0 | B1 | B0 | B1 | B0 | B1 | B0 | B1| B0 | B1 | B0 | B1 | B0 | B1 | B0 | B1|
|M0|0.0 | 0.7 | 0.5 | 0.7 | 0.0 | 0.6 | 0.5 |  0.6 | 0.0 | 0.7 | 0.5 |  0.7 | 0.0 | 0.6 | 0.5 |  0.6|
|M1| 1.0 | 0.2 | 0.1 |  0.1 | 1.0 | 0.3 | 0.2 |  0.2 | 1.0 | 0.2 | 0.1 |  0.1 | 1.0 | 0.3 | 0.2 |  0.2 |
|M2| 0.0 | 0.1 | 0.4 |  0.2 | 0.0 | 0.1 | 0.3 |  0.2 | 0.0 | 0.1 | 0.4 |  0.2 | 0.0 | 0.1 | 0.3 |  0.2 |

Using the above table, define the CPD (sorry about all the typing:). 

In [10]:
CDP_M = TabularCPD(variable="M", variable_card=3, values=[
            [0.0, 0.7, 0.5, 0.7, 0.0, 0.6, 0.5, 0.6, 0.0, 0.7, 0.5, 0.7, 0.0, 0.6, 0.5, 0.6],
            [1.0, 0.2, 0.1, 0.1, 1.0, 0.3, 0.2, 0.2, 1.0, 0.2, 0.1, 0.1, 1.0, 0.3, 0.2, 0.2],
            [0.0, 0.1, 0.4, 0.2, 0.0, 0.1, 0.3, 0.2, 0.0, 0.1, 0.4, 0.2, 0.0, 0.1, 0.3, 0.2]],
                   evidence=["CW","BW","C","B"], evidence_card=[2,2,2,2])
print(CDP_M)


╒═════╤══════╤══════╤══════╤══════╤══════╤══════╤══════╤══════╤══════╤══════╤══════╤══════╤══════╤══════╤══════╤══════╕
│ CW  │ CW_0 │ CW_0 │ CW_0 │ CW_0 │ CW_0 │ CW_0 │ CW_0 │ CW_0 │ CW_1 │ CW_1 │ CW_1 │ CW_1 │ CW_1 │ CW_1 │ CW_1 │ CW_1 │
├─────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┤
│ BW  │ BW_0 │ BW_0 │ BW_0 │ BW_0 │ BW_1 │ BW_1 │ BW_1 │ BW_1 │ BW_0 │ BW_0 │ BW_0 │ BW_0 │ BW_1 │ BW_1 │ BW_1 │ BW_1 │
├─────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┤
│ C   │ C_0  │ C_0  │ C_1  │ C_1  │ C_0  │ C_0  │ C_1  │ C_1  │ C_0  │ C_0  │ C_1  │ C_1  │ C_0  │ C_0  │ C_1  │ C_1  │
├─────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┤
│ B   │ B_0  │ B_1  │ B_0  │ B_1  │ B_0  │ B_1  │ B_0  │ B_1  │ B_0  │ B_1  │ B_0  │ B_1  │ B_0  │ B_1  │ B_0  │ B_1  │
├─────┼──────┼──────┼──────┼──────┼─────

To complete your belief network, use the `add_cpds` method. 

> **Hint:** Before going any further make sure you apply the `check_model` method to your complete model. 

In [11]:
m_model.add_cpds(CDP_B,CDP_BW,CDP_C,CDP_CW,CDP_M)
m_model.check_model()

True

Next that the independencies of all the variables in your model are correct using the `local_independencies` method. 

In [21]:
print(m_model.local_independencies(['B','BW','C','CW','M']))
m_model.get_cpds()  #just check cpd as well


(B _|_ C, CW)
(BW _|_ C, CW | B)
(C _|_ BW, B)
(CW _|_ BW, B | C)


[<TabularCPD representing P(B:2) at 0x20a0d39fdd8>,
 <TabularCPD representing P(BW:2 | B:2) at 0x20a0e4552b0>,
 <TabularCPD representing P(C:2) at 0x20a0d39fda0>,
 <TabularCPD representing P(CW:2 | C:2) at 0x20a0e4552e8>,
 <TabularCPD representing P(M:3 | CW:2, BW:2, C:2, B:2) at 0x20a0e455518>]

**Question:** Is your graph an I-map of the distribution and why?

ANS: Yes. This graph is an I-map because our distribution's the independence is

$$p(B,C,BW,CW,M) = p(B)\ p(C)\ p(BW\ |\ B)\ p(CW\ |\ C)\ p(M\ |\ BW,CW, C, B)$$  

And our graph's independeces are 

(B _|_ C, CW)
(BW _|_ C, CW | B)
(C _|_ BW, B)
(CW _|_ BW, B | C)

It could represents the independencies of the distribution.


Now you are ready to perform inference on your model. 

For the first query determine the marginal probability of weapon choice for the butler (with no evidence) using the variable elimination method. 

In [27]:
from pgmpy.inference import VariableElimination
m_model_inference = VariableElimination(m_model)


In [31]:
# m_model_inference.induced_width(['B','BW','C','CW','M'])
qur_BW = m_model_inference.query(variables=['BW'])
print(qur_BW['BW'])

╒══════╤═══════════╕
│ BW   │   phi(BW) │
╞══════╪═══════════╡
│ BW_0 │    0.8200 │
├──────┼───────────┤
│ BW_1 │    0.1800 │
╘══════╧═══════════╛


C:\Users\genie\Anaconda3\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:598: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi1.values = phi1.values[slice_]


Next make query to find the marginal probability of weapon choice with evidence the butler is a murderer.

In [35]:
qur_BWB0 = m_model_inference.query(variables=['BW'], evidence={'B':1})
print(qur_BWB0['BW'])

╒══════╤═══════════╕
│ BW   │   phi(BW) │
╞══════╪═══════════╡
│ BW_0 │    0.9000 │
├──────┼───────────┤
│ BW_1 │    0.1000 │
╘══════╧═══════════╛


C:\Users\genie\Anaconda3\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:598: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi1.values = phi1.values[slice_]


How are the marginal probabilities change by introducing the evidence? Why do you think this is the case?

ANS: The marginal probabilities of weapon choice of butler with only knife is higer
when given evidence that the bulter is the murderer.
It is reasonable because of 

| | B0| B1 |
|---|---|---|
|BW0 | 0.5 | 0.9 | 
|BW1 | 0.5 | 0.1 | 

This value we have. 
According to this table, when the bulter is the murderer (B1), BW 0 = 0.9 , BW1 = 0.1

Now, turn your attention to the variable M. For your first query on this variable use evidence that the cook and the butler selected knife and poison as the weapon. 

In [36]:
qur_M = m_model_inference.query(variables=["M"], evidence={'CW':1, 'BW':1})
print(qur_M['M'])

╒═════╤══════════╕
│ M   │   phi(M) │
╞═════╪══════════╡
│ M_0 │   0.3708 │
├─────┼──────────┤
│ M_1 │   0.5056 │
├─────┼──────────┤
│ M_2 │   0.1236 │
╘═════╧══════════╛


C:\Users\genie\Anaconda3\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:598: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi1.values = phi1.values[slice_]
C:\Users\genie\Anaconda3\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:586: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi.values = phi.values[slice_]


Now perform a query on the M table with evidence that the cook and the butler selected a knife alone as the weapon.

In [62]:
qur_M2 = m_model_inference.query(variables=["M"], evidence={'CW':0, 'BW':0})
print(qur_M2['M'])


╒═════╤══════════╕
│ M   │   phi(M) │
╞═════╪══════════╡
│ M_0 │   0.6271 │
├─────┼──────────┤
│ M_1 │   0.2572 │
├─────┼──────────┤
│ M_2 │   0.1157 │
╘═════╧══════════╛


C:\Users\genie\Anaconda3\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:586: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi.values = phi.values[slice_]
C:\Users\genie\Anaconda3\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:598: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi1.values = phi1.values[slice_]


How can you best explain the change in the marginal distributions given the different evidence?

ANS:  We can explain by the table

| | p | p | p | p | p | p | p | p | p | p | p | p |  p | p | p | p |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|| CW0 | CW0 | CW0 | CW0 | CW0 | CW0 | CW0 | CW0 | CW1 | CW1 | CW1 | CW1 | CW1 | CW1 | CW1 | CW1 |
|| BW0 | BW0 | BW0 | BW0 | BW1 | BW1 | BW1 | BW1 | BW0 | BW0 | BW0 | BW0 | BW1 | BW1 | BW1 | BW1 |
|| C0 | C0 | C1 | C1 | C0 | C0 | C1 | C1| C0 | C0 | C1 | C1| C0 | C0 | C1 | C1 |
|| B0 | B1 | B0 | B1 | B0 | B1 | B0 | B1| B0 | B1 | B0 | B1 | B0 | B1 | B0 | B1|
|M0|0.0 | 0.7 | 0.5 | 0.7 | 0.0 | 0.6 | 0.5 |  0.6 | 0.0 | 0.7 | 0.5 |  0.7 | 0.0 | 0.6 | 0.5 |  0.6|
|M1| 1.0 | 0.2 | 0.1 |  0.1 | 1.0 | 0.3 | 0.2 |  0.2 | 1.0 | 0.2 | 0.1 |  0.1 | 1.0 | 0.3 | 0.2 |  0.2 |
|M2| 0.0 | 0.1 | 0.4 |  0.2 | 0.0 | 0.1 | 0.3 |  0.2 | 0.0 | 0.1 | 0.4 |  0.2 | 0.0 | 0.1 | 0.3 |  0.2 |

This explain is not accurate, becuase marginal distributions are caluated by conditional distribution, but intuitively, 

the marginal distribution of CW0 and BW0 is

0.0+0.7+0.5+0.7 | 1.0+0.2+0.1+0.1 | 0.0+0.1+0.4+0.2

= 1.9 | 1.4 | 0.7    -> normalize = 0.475 | 0.35 | 0.175

CW1 and BW1 is  0.0+0.6+0.5+0.6 / 1.0+0.3+0.2+0.2 / 0.0+0.1+0.3+0.2

= 1.7 | 1.7 | 0.6    -> normalize = 0.425 | 0.425 | 0.15

approximately, we can see difference ratio between given CW:0, BW:0 and given CW:1, BW:1


Now, perform a query on the M variable with evidence that the cook was not involved and the butler's choice of weapon was the knife alone.  

In [41]:
qur_M3 = m_model_inference.query(variables=['M'], evidence={'C':0, 'BW':0})
print(qur_M3['M'])

╒═════╤══════════╕
│ M   │   phi(M) │
╞═════╪══════════╡
│ M_0 │   0.6146 │
├─────┼──────────┤
│ M_1 │   0.2976 │
├─────┼──────────┤
│ M_2 │   0.0878 │
╘═════╧══════════╛


C:\Users\genie\Anaconda3\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:586: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi.values = phi.values[slice_]
C:\Users\genie\Anaconda3\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:598: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi1.values = phi1.values[slice_]


For the final set of queries you will use the belief propagation method to your DAG. Perform belief propagation and print the factors.

In [56]:
from pgmpy.inference import BeliefPropagation
m_belief = BeliefPropagation(m_model)
m_belief.factors

C:\Users\genie\Anaconda3\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:586: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi.values = phi.values[slice_]
C:\Users\genie\Anaconda3\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:598: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi1.values = phi1.values[slice_]


defaultdict(list,
            {'B': [<DiscreteFactor representing phi(B:2) at 0x20a0e60f550>,
              <DiscreteFactor representing phi(BW:2, B:2) at 0x20a0e60f780>,
              <DiscreteFactor representing phi(M:3, CW:2, BW:2, C:2, B:2) at 0x20a0e60fa20>],
             'BW': [<DiscreteFactor representing phi(BW:2, B:2) at 0x20a0e60f780>,
              <DiscreteFactor representing phi(M:3, CW:2, BW:2, C:2, B:2) at 0x20a0e60fa20>],
             'C': [<DiscreteFactor representing phi(C:2) at 0x20a0e60f940>,
              <DiscreteFactor representing phi(CW:2, C:2) at 0x20a0e60fa90>,
              <DiscreteFactor representing phi(M:3, CW:2, BW:2, C:2, B:2) at 0x20a0e60fa20>],
             'CW': [<DiscreteFactor representing phi(CW:2, C:2) at 0x20a0e60fa90>,
              <DiscreteFactor representing phi(M:3, CW:2, BW:2, C:2, B:2) at 0x20a0e60fa20>],
             'M': [<DiscreteFactor representing phi(M:3, CW:2, BW:2, C:2, B:2) at 0x20a0e60fa20>]})

In [57]:
# for key in m_belief.clique_beliefs.keys():
#     print(m_belief.clique_beliefs[key])


How many cliques are in the graph and what is the maximum span (number of nodes)? 

ANS: It has [BW,B] , [M,CW,BW,C,B] , [CW,C].  3 cliques, maximum 5 span.


Now query the BW and M variables using evidence that the cook is not a murderer.

In [60]:
belief_query = m_belief.query(variables=['BW','M'], evidence={'C':0})
print(belief_query['BW'])
print(belief_query['M'])

╒══════╤═══════════╕
│ BW   │   phi(BW) │
╞══════╪═══════════╡
│ BW_0 │    0.8200 │
├──────┼───────────┤
│ BW_1 │    0.1800 │
╘══════╧═══════════╛
╒═════╤══════════╕
│ M   │   phi(M) │
╞═════╪══════════╡
│ M_0 │   0.5520 │
├─────┼──────────┤
│ M_1 │   0.3680 │
├─────┼──────────┤
│ M_2 │   0.0800 │
╘═════╧══════════╛


Compare these marginal distributions to the marginal distributions you obtained with simple variable elimination. Is this result what you expected? 

ANS: Yes, this result is quite understandable. little bit less M_0 but higher M_1. 


There are many other possible queries you can perform on this model. You may wish to explore some more of these. 